# Курсовая

## Вариант 19  
Решить краевую задачу для обыкновенного дифференциального уравнения второго порядка  
$$
\begin{equation*}
\begin{cases}
xy''-(2x+1)y'+(x+1)y=0\\
y'(1)=3e\\
py(2)+qy'(2)=0
\end{cases}
\end{equation*}
$$  
где *p* и *q* удовлетворяют системе уравнений
$$
\begin{equation*}
\begin{cases}
sin(p+0.5) + q = 0\\
cos(q-0.5) - 1.1p = 3
\end{cases}
\end{equation*}
$$

#### Решение задачи  
Решение задачи будет заключаться в решениях СНАУ *методом Ньютона* и краевой задачи для ОДУ второго порядка *методом стрельбы*.  
